# As long as the excel structure of the portfolio remains constant, it may freely be updated and the tool will update automatically

In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
portfolio=pd.read_csv("//lux2//d_actuaire/ModelesDev/pySolvencyMonitoring/assets/main_file_IDS_2020Q4.csv")

In [3]:
portfolio.drop(portfolio[portfolio["INPUT_TBL"]=="residual_input"].index,inplace=True)
portfolio=portfolio.loc[(portfolio["CART_ASSET_CLASS_L1_CODE"]=="FI") & (portfolio["ASSET_TYPE"]=="FI")]

In [4]:
portfolio=portfolio[["CREDIT_RATING","ISSUER_BE","COUNTRY_OF_RISK","MARKET_VAL_BE"]]

In [5]:
total_exposure=portfolio["MARKET_VAL_BE"].sum()

In [6]:
portfolio["MARKET_VAL_BE"]=portfolio["MARKET_VAL_BE"].apply(lambda x : (x/total_exposure)*100)
portfolio

CREDIT_RATING ISSUER_BE COUNTRY_OF_RISK  MARKET_VAL_BE
84             AA        GO              BE       1.421976
87             AA        GO              BE       1.751296
88             AA        GO              BE       1.756864
89            BBB        CO              GB       0.500294
90             AA        GO              BE       2.635194
..            ...       ...             ...            ...
284           BBB        CO              FR       0.441781
287             A        CO              US       0.646217
288            AA        GO              FR       1.473092
290           BBB        CO              FR       0.734802
291           BBB        CO              ES       0.223384

[110 rows x 4 columns]

In [7]:
portfolio["SPREAD_CATEGORY"]=""
portfolio.loc[portfolio["ISSUER_BE"]=="GO","SPREAD_CATEGORY"]=portfolio.loc[portfolio["ISSUER_BE"]=="GO","COUNTRY_OF_RISK"].apply(lambda x : "EUR-SPREAD-GOV-"+x)
portfolio.loc[portfolio["ISSUER_BE"]=="CO","SPREAD_CATEGORY"]=portfolio.loc[portfolio["ISSUER_BE"]=="CO","CREDIT_RATING"].apply(lambda x : "EUR-SPREAD-CORP-"+x)
portfolio

CREDIT_RATING ISSUER_BE COUNTRY_OF_RISK  MARKET_VAL_BE  \
84             AA        GO              BE       1.421976   
87             AA        GO              BE       1.751296   
88             AA        GO              BE       1.756864   
89            BBB        CO              GB       0.500294   
90             AA        GO              BE       2.635194   
..            ...       ...             ...            ...   
284           BBB        CO              FR       0.441781   
287             A        CO              US       0.646217   
288            AA        GO              FR       1.473092   
290           BBB        CO              FR       0.734802   
291           BBB        CO              ES       0.223384   

         SPREAD_CATEGORY  
84     EUR-SPREAD-GOV-BE  
87     EUR-SPREAD-GOV-BE  
88     EUR-SPREAD-GOV-BE  
89   EUR-SPREAD-CORP-BBB  
90     EUR-SPREAD-GOV-BE  
..                   ...  
284  EUR-SPREAD-CORP-BBB  
287    EUR-SPREAD-CORP-A  
288    EUR-SPREAD-GOV-FR  
290  EUR-SPREAD-CORP-BBB  
291  EUR-SPREAD-CORP-BBB  

[110 rows x 5 columns]

In [18]:
portfolio=portfolio.groupby(["SPREAD_CATEGORY"]).sum(["MARKET_VAL_BE"])
portfolio=portfolio.sort_values(by="MARKET_VAL_BE",ascending=False)
portfolio=portfolio.reset_index()
portfolio

SPREAD_CATEGORY  MARKET_VAL_BE
0     EUR-SPREAD-GOV-BE      29.195021
1   EUR-SPREAD-CORP-BBB      16.188947
2     EUR-SPREAD-GOV-DE      14.410424
3     EUR-SPREAD-GOV-NL      12.010450
4     EUR-SPREAD-GOV-FR       8.783370
5     EUR-SPREAD-CORP-A       7.446025
6     EUR-SPREAD-GOV-IT       2.911195
7     EUR-SPREAD-GOV-ES       2.635049
8     EUR-SPREAD-GOV-NO       2.611758
9    EUR-SPREAD-CORP-AA       2.207068
10  EUR-SPREAD-CORP-AAA       0.457797
11   EUR-SPREAD-GOV-SNE       0.455115
12    EUR-SPREAD-GOV-DK       0.445587
13    EUR-SPREAD-GOV-FI       0.242193

In [9]:
def predefined_percentage(portfolio,percentage_exposure):

    counter=1
    suma=0

    for x in portfolio["MARKET_VAL_BE"]:
        if suma <= percentage_exposure:
            suma = suma + x
            counter = counter + 1
        else:
            break
    
    portfolio=portfolio.iloc[:counter]
        
    return portfolio

In [19]:
predefined_percentage(portfolio,80)

SPREAD_CATEGORY  MARKET_VAL_BE
0    EUR-SPREAD-GOV-BE      29.195021
1  EUR-SPREAD-CORP-BBB      16.188947
2    EUR-SPREAD-GOV-DE      14.410424
3    EUR-SPREAD-GOV-NL      12.010450
4    EUR-SPREAD-GOV-FR       8.783370
5    EUR-SPREAD-CORP-A       7.446025

# Here the credit_spreads dataframe You will replace with the one I can not access

In [10]:
keys=["SPREAD_CATEGORY","SPREAD_REFERENCE","SPREAD_ACTUAL"]
value_1=np.array([i for i in predefined_percentage(portfolio,80).index.tolist()])
value_2=np.array([np.random.uniform(0.01,0.03) for i in value_1 ])
value_3=value_2 + 0.005
values=np.concatenate((value_1,value_2,value_3))
values=np.array_split(values,len(values)/len(value_1))

In [11]:
for i in np.arange(1,len(values)):
    values[i]=values[i].astype("float64")

In [12]:
dictionary=dict(zip(keys,values))
credit_spreads=pd.DataFrame(dictionary,index=range(0,len(value_1)))
credit_spreads

SPREAD_CATEGORY  SPREAD_REFERENCE  SPREAD_ACTUAL
0    EUR-SPREAD-GOV-BE          0.021206       0.026206
1  EUR-SPREAD-CORP-BBB          0.013972       0.018972
2    EUR-SPREAD-GOV-DE          0.028193       0.033193
3    EUR-SPREAD-GOV-NL          0.014030       0.019030
4    EUR-SPREAD-GOV-FR          0.020045       0.025045
5    EUR-SPREAD-CORP-A          0.014990       0.019990

In [13]:
def subtraction(a,b):
    return a-b

In [14]:
credit_spreads["SPREAD_CHANGE"]=np.vectorize(subtraction)(credit_spreads["SPREAD_ACTUAL"],credit_spreads["SPREAD_REFERENCE"])
credit_spreads

SPREAD_CATEGORY  SPREAD_REFERENCE  SPREAD_ACTUAL  SPREAD_CHANGE
0    EUR-SPREAD-GOV-BE          0.021206       0.026206          0.005
1  EUR-SPREAD-CORP-BBB          0.013972       0.018972          0.005
2    EUR-SPREAD-GOV-DE          0.028193       0.033193          0.005
3    EUR-SPREAD-GOV-NL          0.014030       0.019030          0.005
4    EUR-SPREAD-GOV-FR          0.020045       0.025045          0.005
5    EUR-SPREAD-CORP-A          0.014990       0.019990          0.005

In [22]:
final_spreads=predefined_percentage(portfolio,80).merge(credit_spreads,how="inner",on="SPREAD_CATEGORY")
final_spreads

SPREAD_CATEGORY  MARKET_VAL_BE  SPREAD_REFERENCE  SPREAD_ACTUAL  \
0    EUR-SPREAD-GOV-BE      29.195021          0.021206       0.026206   
1  EUR-SPREAD-CORP-BBB      16.188947          0.013972       0.018972   
2    EUR-SPREAD-GOV-DE      14.410424          0.028193       0.033193   
3    EUR-SPREAD-GOV-NL      12.010450          0.014030       0.019030   
4    EUR-SPREAD-GOV-FR       8.783370          0.020045       0.025045   
5    EUR-SPREAD-CORP-A       7.446025          0.014990       0.019990   

   SPREAD_CHANGE  
0          0.005  
1          0.005  
2          0.005  
3          0.005  
4          0.005  
5          0.005

In [ ]:
#dataframe.pivot look up the API

In [6]:
non_clean_ratings=[]
countries=[]
indexlist=[]

for i in portfolio["CREDIT_RATING"]:
    if i not in non_clean_ratings:
        non_clean_ratings.append(i)
        
non_clean_ratings.append("Total Exposure")

for i in portfolio["COUNTRY_OF_RISK"]:
    if i not in countries:
        countries.append(i)
        
ratings=sorted([x for x in non_clean_ratings if str(x)!="nan" if str(x)!="NaN"])
indexratings=ratings*2


for i in np.arange(len(ratings)*2):
    if i<len(ratings):
        indexlist.append("CO")
    else:
        indexlist.append("GO")

In [6]:
empty=np.array(())
count=0

for i in np.arange(len(countries)):
    for j in np.arange(len(indexratings)):
            
        if (j<(len(ratings)-1)):
            
                rating_exposure=np.array([np.round((np.true_divide((portfolio.loc[(portfolio["COUNTRY_OF_RISK"]==countries[i]) & (portfolio["CREDIT_RATING"]==ratings[j]) & (portfolio["ISSUER_BE"]=="GO"),"MARKET_VAL_BE"].sum()),total_exposure)*100),6)])
                empty=np.concatenate((empty,rating_exposure))
            
        elif (j==(len(ratings)-1)):
                
                temporary_1=np.array([np.sum(empty[count:count+(len(ratings)-1)])])
                empty=np.concatenate((empty,temporary_1))
                temporary_1=np.array(())
            
        elif (((j>len(ratings)-1) and j<len(indexratings)-1)):
            
                rating_exposure=np.array([np.round((np.true_divide((portfolio.loc[(portfolio["COUNTRY_OF_RISK"]==countries[i]) & (portfolio["CREDIT_RATING"]==indexratings[j]) & (portfolio["ISSUER_BE"]=="CO"),"MARKET_VAL_BE"].sum()),total_exposure)*100),6)])
                empty=np.concatenate((empty,rating_exposure))
            
        else:
                temporary_2=np.array([np.sum(empty[count+(len(ratings)):count+(j)])])
                empty=np.concatenate((empty,temporary_2))
                temporary_2=np.array(())
        
    count=(count+len(indexratings))


In [7]:
results=np.array(np.array_split(empty,(len(empty)/len(indexratings))))

In [8]:
data_dictionary=dict(zip(countries,results))
table=pd.DataFrame(data_dictionary,index=[indexlist,indexratings])

In [9]:
table

LU         BE        DE         NL   M6        GB  \
CO A               0.000000   0.352548  0.000000   0.000000  0.0  0.000000   
   AA              0.000000  17.473041  0.566032   0.000000  0.0  0.000000   
   AAA             0.000000   0.000000  8.232534   7.333214  0.0  0.000000   
   BBB             0.000000   0.000000  0.000000   0.000000  0.0  0.000000   
   NR              0.000000   0.000000  0.000000   0.000000  0.0  0.000000   
   Total Exposure  0.000000  17.825589  8.798566   7.333214  0.0  0.000000   
GO A               1.709670   0.273299  0.000000   0.320864  0.0  0.000000   
   AA              0.410266   0.000000  0.000000  10.574457  0.0  0.000000   
   AAA             0.000000   0.000000  0.000000   0.279517  0.0  0.000000   
   BBB             0.000000   0.000000  0.895585   1.814671  0.0  1.720803   
   NR              1.776213   0.074183  0.182973   0.000000  0.0  0.000000   
   Total Exposure  3.896149   0.347482  1.078558  12.989509  0.0  1.720803   

                         FR        SE        AT   M7  ...        US        IT  \
CO A               1.372123  0.000000  0.000000  0.0  ...  0.000000  0.000000   
   AA              3.990735  0.000000  0.000000  0.0  ...  0.000000  0.000000   
   AAA             0.000000  0.000000  0.000000  0.0  ...  0.000000  0.000000   
   BBB             0.000000  0.000000  0.000000  0.0  ...  0.000000  1.777487   
   NR              0.000000  0.000000  0.000000  0.0  ...  0.000000  0.000000   
   Total Exposure  5.362858  0.000000  0.000000  0.0  ...  0.000000  1.777487   
GO A               1.802018  0.401014  0.000000  0.0  ...  1.026482  0.000000   
   AA              0.273525  0.000000  0.000000  0.0  ...  0.400657  0.000000   
   AAA             0.000000  0.000000  0.000000  0.0  ...  0.000000  0.000000   
   BBB             2.091610  0.478061  0.108424  0.0  ...  0.140160  1.100939   
   NR              0.000000  0.000000  0.000000  0.0  ...  0.000000  0.000000   
   Total Exposure  4.167153  0.879075  0.108424  0.0  ...  1.567299  1.100939   

                         M1        NO        ES        DK        ZA       SNE  \
CO A               0.000000  0.858948  1.608881  0.000000  0.000000  0.000000   
   AA              0.000000  0.735711  0.000000  0.000000  0.000000  0.277879   
   AAA             0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   BBB             0.000000  0.000000  0.000000  0.272062  0.000000  0.000000   
   NR              0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   Total Exposure  0.000000  1.594659  1.608881  0.272062  0.000000  0.277879   
GO A               0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   AA              0.000000  0.000000  0.383519  0.000000  0.000000  0.000000   
   AAA             0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   BBB             0.000000  0.000000  0.136391  0.399703  0.199187  0.000000   
   NR              2.152901  0.000000  0.000000  0.000000  0.000000  0.000000   
   Total Exposure  2.152901  0.000000  0.519910  0.399703  0.199187  0.000000   

                         AU        CH  
CO A               0.000000  0.000000  
   AA              0.000000  0.000000  
   AAA             0.000000  0.000000  
   BBB             0.000000  0.000000  
   NR              0.000000  0.000000  
   Total Exposure  0.000000  0.000000  
GO A               0.139862  0.297828  
   AA              0.000000  0.000000  
   AAA             0.000000  0.000000  
   BBB             0.000000  0.000000  
   NR              0.000000  0.000000  
   Total Exposure  0.139862  0.297828  

[12 rows x 21 columns]

**Total corporate exposure in %

In [10]:
table.loc[("CO","Total Exposure"),:].sum()

44.999070999999994

**Total government exposure in %

In [11]:
table.loc[("GO","Total Exposure"),:].sum()

32.363723

**Total Fixed Income exposure in %

In [12]:
table.loc[("CO","Total Exposure"),:].sum() + table.loc[("GO","Total Exposure"),:].sum()

77.362794